In [1]:
import numpy as np
import pandas as pd
# Warnings
import warnings
warnings.filterwarnings('ignore')
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data1 = pd.read_csv("Strain plot_expt1.csv")
data2 = pd.read_csv("Strain plot_expt2.csv")
data3 = pd.read_csv("Strain plot_expt3.csv")
data_t = pd.read_csv("Strain plot_expt4.csv")

In [3]:
data1.shape,data2.shape,data3.shape,data_t.shape

((52, 13), (52, 13), (52, 13), (52, 13))

In [4]:
data = pd.concat([data1,data2,data3,data_t], axis=0)

In [5]:
data.corr()

,Time,A,B,C,D,X_P,Y_P,P,Q,R,S,X_D,Y_D
Time,1.000000,-0.219849,-0.219590,0.064700,0.341070,-0.018525,-0.180866,0.056601,0.331120,-0.214708,-0.226262,-0.020095,0.169980
A,-0.219849,1.000000,0.104762,-0.170968,0.093912,-0.001526,0.038796,-0.166868,0.118263,0.971879,0.149892,-0.000198,0.024297
B,-0.219590,0.104762,1.000000,0.099896,-0.173143,0.029339,0.001808,0.114124,-0.174808,0.131087,0.979012,0.037531,0.000016
C,0.064700,-0.170968,0.099896,1.000000,0.080097,-0.012058,-0.071751,0.971934,0.094917,-0.181662,0.107128,0.015996,0.003239
D,0.341070,0.093912,-0.173143,0.080097,1.000000,-0.029435,0.011977,0.141578,0.960186,0.105860,-0.166511,-0.041547,-0.010843
X_P,-0.018525,-0.001526,0.029339,-0.012058,-0.029435,1.000000,0.000330,0.013906,0.043626,-0.002025,-0.044205,-0.983417,-0.001250
Y_P,-0.180866,0.038796,0.001808,-0.071751,0.011977,0.000330,1.000000,0.022980,0.007192,-0.053706,-0.027372,-0.000470,-0.982777
P,0.056601,-0.166868,0.114124,0.971934,0.141578,0.013906,0.022980,1.000000,0.163078,-0.177386,0.127425,-0.016022,-0.085359
Q,0.331120,0.118263,-0.174808,0.094917,0.960186,0.043626,0.007192,0.163078,1.000000,0.139905,-0.168543,-0.096566,-0.011122
R,-0.214708,0.971879,0.131087,-0.181662,0.105860,-0.002025,-0.053706,-0.177386,0.139905,1.000000,0.185642,0.001190,0.113767


In [22]:
trainX = data[[i for i in list(data.columns) if i not in ["Time","Y_P","Y_D","X_D"]]].values
trainY = data[[i for i in list(data.columns) if i not in ["Time","X_P","Y_D","X_D"]]].values
targetX = data.iloc[:,-2].values.reshape((-1,1))
targetY = data.iloc[:,-1].values.reshape((-1,1))

In [23]:
trainX.shape,trainY.shape,targetX.shape,targetY.shape

((208, 9), (208, 9), (208, 1), (208, 1))

In [24]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
scY= StandardScaler()

trainX_scaled = scX.fit_transform(trainX)
trainY_scaled = scY.fit_transform(trainY)

In [25]:
trainX_scaled.shape

(208, 9)

In [26]:
# Evaluate using Cross Validation
from sklearn.model_selection import cross_val_predict,cross_val_score,KFold
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [27]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

regressorX = RandomForestRegressor()
regressorY = RandomForestRegressor()

scoreX = cross_val_score(regressorX, trainX_scaled, targetX, cv=kfold)
scoreY = cross_val_score(regressorY, trainY_scaled, targetY, cv=kfold)

In [28]:
scoreX.mean(),scoreY.mean()

(0.976174597337789, 0.9785458367503617)

In [29]:
# decision tree
from sklearn.tree import DecisionTreeRegressor

regressor1X = DecisionTreeRegressor()
regressor1Y = DecisionTreeRegressor()

score1X = cross_val_score(regressor1X, trainX_scaled, targetX, cv=kfold)
score1Y = cross_val_score(regressor1Y, trainY_scaled, targetY, cv=kfold)

In [14]:
score1X.mean(),score1Y.mean()

(0.9468124293823246, 0.9698503344872924)

In [15]:
# linear regression
from sklearn import linear_model

regressor2X = linear_model.LinearRegression()
regressor2Y = linear_model.LinearRegression()

score2X = cross_val_score(regressor2X, trainX_scaled, targetX, cv=kfold)
score2Y = cross_val_score(regressor2Y, trainY_scaled, targetY, cv=kfold)

In [16]:
score2X.mean(),score2Y.mean()

(0.9652623296822934, 0.964923173276502)

In [17]:
# svm
from sklearn.svm import SVR

regressor3X = SVR(kernel = 'rbf')
regressor3Y = SVR(kernel = 'rbf')

score3X = cross_val_score(regressor3X, trainX_scaled, targetX, cv=kfold)
score3Y = cross_val_score(regressor3Y, trainY_scaled, targetY, cv=kfold)

In [18]:
score3X.mean(),score3Y.mean()

(0.40776260912827944, 0.39530713153293856)

In [19]:
# Polynomial regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

polynomial_featuresX= PolynomialFeatures(degree=2)
polynomial_featuresY= PolynomialFeatures(degree=2)


modelX = LinearRegression()
modelY = LinearRegression() 





In [20]:
scoresX=[]
scoresY=[]

from sklearn.metrics import mean_squared_error,r2_score,accuracy_score
from sklearn.model_selection import KFold 
kf = KFold(n_splits=10,shuffle=True , random_state=seed)

for train_index, test_index in kf.split(trainX):
    X_train, X_test = trainX[train_index], trainX[test_index]
    y_train, y_test = targetX[train_index], targetX[test_index]
    
    X_train = polynomial_featuresX.fit_transform(X_train)
    X_test = polynomial_featuresX.transform(X_test)
    
    modelX = modelX.fit(X_train, y_train)
    y_predX = modelX.predict(X_test)

    
    scoresX.append(r2_score(y_test,y_predX))
    
    
scoresX = np.array(scoresX)



for train_index, test_index in kf.split(trainY):
    X_train, X_test = trainY[train_index], trainY[test_index]
    y_train, y_test = targetY[train_index], targetY[test_index]
    
    X_train = polynomial_featuresY.fit_transform(X_train)
    X_test = polynomial_featuresY.transform(X_test)
    
    modelY = modelY.fit(X_train, y_train)
    y_predY = modelY.predict(X_test)

    
    scoresY.append(r2_score(y_test,y_predY))
    
    
scoresY = np.array(scoresY)

In [21]:
scoresX.mean(),scoresY.mean()

(0.9363308098139813, 0.9112203883696829)

In [ ]:
# y_pred_Bottle_X = pd.DataFrame(y_pred)
# y_pred1_Bottle_X = pd.DataFrame(y_pred1)
# y_pred2_Bottle_X = pd.DataFrame(y_pred2)
# y_pred3_Bottle_X = pd.DataFrame(y_pred3)

# Attempt 2

In [32]:
trainX = data[[i for i in list(data.columns) if i in ["P","Q","R","S"]]]
targetX = data.iloc[:,-2].values.reshape((-1,1))

In [34]:
trainX

,P,Q,R,S
0,0.0250,0.0346,0.0700,0.0436
1,0.0730,0.0240,0.0175,0.0350
2,0.0374,0.0630,0.0673,0.0465
3,0.0254,0.0730,0.0275,0.0275
4,0.0240,0.0250,0.0548,0.0650
...,...,...,...,...
47,0.0240,0.0430,0.0640,0.0500
48,0.0360,0.0200,0.0200,0.0400
49,0.0240,0.0750,0.0700,0.0570
50,0.0360,0.0240,0.0300,0.0250


In [35]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
# scY= StandardScaler()

trainX_scaled = scX.fit_transform(trainX)
# trainY_scaled = scY.fit_transform(trainY)

In [36]:
# Evaluate using Cross Validation
from sklearn.model_selection import cross_val_predict,cross_val_score,KFold
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [38]:
score2X

array([-0.24303666, -0.11734017, -0.46151318, -0.06055016, -0.28108146,
       -0.50468158, -0.61443786,  0.00408137, -0.22630967, -0.07167383])